# Not so clean text to tables (pandas fails)

An example where ``import_flatfile_into_database`` does a better job than pandas about converting a flat file to a table.

In [1]:
import random, pandas
text = [ "one","two","three","four","five","six","seven","eight","nine","ten" ]
data = [ { "name": text[random.randint(0,9)], "number": random.randint(0,99)} \
                        for i in range(0,10000) ]
df = pandas.DataFrame(data)
df.head(n=3)

,name,number
0,one,75
1,two,73
2,three,77


In [3]:
df.to_csv("flatfile.txt", sep="\t", encoding="utf8", header=True, index=False)
dfr = pandas.read_csv("flatfile.txt", sep="\t", encoding="utf8")
dfr.head(n=3)

,name,number
0,one,75
1,two,73
2,three,77


Le's assume now we introduce extra tabulations.

In [5]:
datatab = [ {"name": " one\ttab", "number":100 } ] + data
df = pandas.DataFrame(datatab)
df.head(n=3)

,name,number
0,one\ttab,100
1,one,75
2,two,73


In [7]:
df.to_csv("flatfile_tab.txt", sep="\t", encoding="utf8", header=True, index=False)
dfr = pandas.read_csv("flatfile_tab.txt", sep="\t", encoding="utf8")
dfr.head(n=3)

,name,number
0,one\ttab,100
1,one,75
2,two,73


It works well because we use ``pandas`` to save the dataframe, and we use ``pandas`` to restore it. In the file _flatfile_tab.txt_, it looks like ``"on\te"``. ``pandas`` interprets the quotes as a delimiter. However most of the times, the flat file is produced in a different way and the quotes are not present.

In [9]:
with open("flatfile_tab.txt", "r", encoding="utf8") as f:
    content = f.read()
content = content.replace('"','')
with open("flatfile_tab2.txt", "w", encoding="utf8") as f:
    f.write(content)
dfr = pandas.read_csv("flatfile_tab2.txt", sep="\t", encoding="utf8")
dfr.head(n=3)

,name,number
one,tab,100.0
one,75,NaN
two,73,NaN


It failed! Data is not aligned and it did not raise an exception. If we move the extra tab in second position, we get:

In [11]:
datatab = data[:1] + [ {"name": " one\ttab", "number":100 } ] + data[1:]
df = pandas.DataFrame(datatab)
df.to_csv("flatfile_tab_pos2.txt", sep="\t", encoding="utf8", header=True, index=False)
with open("flatfile_tab_pos2.txt","r",encoding="utf8") as f: 
    content = f.read()
content = content.replace('"','')
with open("flatfile_tab_pos2.txt","w",encoding="utf8") as f:
    f.write(content)

In [12]:
dfr = pandas.read_csv("flatfile_tab_pos2.txt", sep="\t", encoding="utf8")
dfr.head(n=3)

ParserError: Error tokenizing data. C error: Expected 2 fields in line 3, saw 3


As suggested in [Python Pandas Error tokenizing data](http://stackoverflow.com/questions/18039057/python-pandas-error-tokenizing-data), we could add the parameter ``error_bad_lines=False`` or ``skiprows=N`` but we would still lose those bad lines. So we use function [import_flatfile_into_database](http://www.xavierdupre.fr/app/pyensae/helpsphinx/pyensae/sql/database_helper.html#sql.database_helper.import_flatfile_into_database).

In [13]:
from pyensae.sql import import_flatfile_into_database
import_flatfile_into_database("flatfile_tab_pos2.db3", "flatfile_tab_pos2.txt")

  TextFile: opening file  flatfile_tab_pos2.txt
  TextFile.guess_columns: processing file  flatfile_tab_pos2.txt
  TextFile: opening file  flatfile_tab_pos2.txt
  TextFile.guess_columns: using  101  lines
  TextFile: closing file  flatfile_tab_pos2.txt
  TextFile.guess_columns: sep  '\t' nb cols 2  bestnb  100  more  {('\t', 1): 100, ('\t', 2): 1}
  TextFile.guess_columns: header  True  columns  {0: ('name', <class 'str'>), 1: ('number', <class 'int'>)}
  compiling ^(?P<name>.*)\t(?P<number>([-]?[1-9][0-9]*?)|(0?))$
  TextFile.guess_columns: regex  ^(?P<name>.*)\t(?P<number>([-]?[1-9][0-9]*?)|(0?))$
  TextFile.guess_columns: header  True  columns  {0: ('name', (<class 'str'>, 10)), 1: ('number', <class 'int'>)}
  [_guess_columns] sep=['\t']
  TextFile: closing file  flatfile_tab_pos2.txt
  [_guess_columns] columns_name=None
   guess with  1001 lines
   count_types  {0: {<class 'str'>: 1000}, 1: {<class 'int'>: 999, <class 'str'>: 1}}
   columns  {0: ('name', <class 'str'>), 1: ('number

'flatfile_tab_pos2'

We check that we got the inserted line in the dataframe:

In [15]:
from pyensae.sql import Database
db = Database("flatfile_tab_pos2.db3")
db.connect()
df = db.to_df("SELECT * FROM flatfile_tab_pos2")
db.close()
df.head()

SQL  'SELECT * FROM flatfile_tab_pos2'


,name,number
0,one,75
1,one\ttab,100
2,two,73
3,three,77
4,six,38
